# 필요한 라이브러리 import

In [1]:
import pandas as pd
import os
import re
import csv
import joblib
import warnings
import numpy as np

# 사전작업

In [2]:
warnings.filterwarnings(action='ignore')

# 포지션 판정 모델 위치
model_path = './models/position_model.pkl'

# 초기 데이터 위치(포지션 판정)
data_path = './datas/FM_DATA'

# 변환된 데이터 위치
target_path = './datas/FM_DATA_MERGED'

# pyspark Mlib 활용에 맞는 데이터셋을 저장할 위치
ml_data_path = './datas/FM_DATA_FOR_ML'

# 포지션 판정 모델 불러오기
position_predict_model=joblib.load(model_path)

# pyspark ml에 사용하기 위해 모든 선수의 데이터를 병합할 데이터 프레임
df_for_pyspark = pd.DataFrame(columns = ['태클 성공률','태클','헤더 시도','헤더 성공%','경기 당 드리블','드리블','패스 시도','패스 %','기회 창출/90','Int/90','달린 거리/90분','Cr A','Cr C/A','슈팅 수','유효 슈팅 %','표준획득승점','포지션'])

# 데이터 전처리

## 한 시즌의 모든 팀의 데이터를 병합하는 함수

In [3]:
def data_merge(path,season):
    # 최종 데이터 프레임 컬럼
    var_list = ['이름', '포지션', '구단', '리그', '임대 기간', '승', '무', '패',
                '출장시간', '경기당 팀 실점/90', '경기당 팀 득점/90', '골', '도움',
                'Cr C/A', 'Cr A', 'FA', '경기 당 드리블', '드리블', '기회 창출/90',
                '유효 슈팅 %', '슈팅 수', 'Int/90', '달린 거리/90분', '헤더 성공%'
                '헤더 시도', '태클 성공률', '태클', '패스 %', '패스 시도', '시즌']
    
    # 모든 변수를 가진 데이터 프레임
    team_df_whole_league = pd.DataFrame(columns=var_list)

    # 특정 경로의 모든 파일 리스트 불러오기(특정리그의 모든 팀들이 저장되어 있음)
    file_list = os.listdir(path)
    
    # 모든 파일을 돌며 -> 각 팀의 통계 자료를 team_df_whole에 합침
    for file in file_list:

        #불러오기
        team_df = pd.read_csv(path+'/'+file,encoding='utf-8')

        # NaN 제거
        team_df = team_df.dropna()

        # 골키퍼는 통계 자료 부족으로 삭제
        team_df = team_df[team_df['포지션'] != 'GK']

        # 현재 구단의 이름을 추출
        team = team_df[team_df['임대 기간']=='-']['구단'].iloc[0]

        # -을 0으로 교체
        team_df = team_df.replace('-',0)

        # 현재 구단에 속해 있으며 출장 시간이 900분을 넘긴 선수만 남김
        team_df = team_df[(team_df['구단']==team) & (team_df['출장시간'].astype('int64')>900)]

        # 시즌 열을 추가
        team_df['시즌'] = season

        # team_df_whole_league 에 연결
        team_df_whole_league=pd.concat([team_df_whole_league,team_df])
        
    # team_df_whole_league의 인덱스를 재설정
    team_df_whole_league = team_df_whole_league.reset_index(drop=True)
    
    # nan은 0으로 대체
    team_df_whole_league = team_df_whole_league.fillna(0)
    
    return team_df_whole_league

## 병합된 시즌 전체 선수 데이터에 포지션을 추정하는 함수

In [4]:
def position_predictor(league_df):
    
    global position_predict_model
    
    # 포지션 추정을 위한 컬럼
    columns_for_ml = ['이름','시즌','구단','승', '무', '패','출장시간','골','도움','태클 성공률','태클','헤더 시도','헤더 성공%','경기 당 드리블','드리블','패스 시도','패스 %','기회 창출/90','Int/90','달린 거리/90분','Cr A','Cr C/A','슈팅 수','유효 슈팅 %']
    
    # 포지션 추정을 위한 df
    df_for_position_ml = league_df[columns_for_ml]

    # 데이터 전처리
    df_for_position_ml['Cr C/A'] = df_for_position_ml['Cr C/A'].str.strip('%')

    df_for_position_ml['달린 거리/90분'] = df_for_position_ml['달린 거리/90분'].str.strip('km')

    df_for_position_ml['태클 성공률'] = df_for_position_ml['태클 성공률'].str.strip('%')

    df_for_position_ml['헤더 성공%'] = df_for_position_ml['헤더 성공%'].str.strip('%')

    df_for_position_ml['패스 %'] = df_for_position_ml['패스 %'].str.strip('%')

    df_for_position_ml['유효 슈팅 %'] = df_for_position_ml['유효 슈팅 %'].str.strip('%')

    df_for_position_ml= df_for_position_ml.fillna(0)
    
    # 판정 결과 저장(포지션 열)
    df_for_position_ml['포지션'] = position_predict_model.predict(df_for_position_ml.iloc[:,7:])
    
    return df_for_position_ml

## 포지션 추정된 데이터 전처리 및 변환하는 함수

In [5]:
def data_cleaner(df):
    
    # 판정 후 추가 데이터 전처리 진행,형변환
    new_df = df.astype({'승':np.float,'무':np.float,'패':np.float,'태클':np.float,'출장시간':np.float,'헤더 시도':np.float,'헤더 시도':np.float,'패스 시도':np.float,'Cr A':np.float,'슈팅 수':np.float})

    # 경기 수 * 경기 당 태클 수로 총 태클 수 구하기
    new_df['태클'] = (new_df['승']+new_df['무']+new_df['패'])*new_df['태클']
    new_df = new_df.astype({'태클':int})

    # 총 태클수를 900분으로 표준화 시킴
    new_df['태클'] = np.round(new_df['태클']/new_df['출장시간']*900,2)

    # 승점 또한 900분으로 표준화
    new_df['표준획득승점'] = np.round((new_df['승']*3 + new_df['무'])/new_df['출장시간']*900,2)

    # 총 시도와 관련된 변수들 전부 900분 으로 표준화
    new_df['패스 시도'] = np.round(new_df['패스 시도']/new_df['출장시간']*900,2)
    new_df['헤더 시도'] = np.round(new_df['헤더 시도']/new_df['출장시간']*900,2)
    new_df['슈팅 수'] = np.round(new_df['슈팅 수']/new_df['출장시간']*900,2)
    new_df['Cr A'] = np.round(new_df['Cr A']/new_df['출장시간']*900,2)
    
    return new_df

# 시즌 별, 리그 단위로 데이터 병합 및 저장

In [6]:
# D:/FM_DATA 내의 목록 가져오기
folder_list = os.listdir(data_path)

In [7]:
# 시즌을 추출할 정규표현식(숫자)
season_re = re.compile('\d+')

# 리그를 추출할 정규표현식(영어+숫자) 예) 프랑스 Ligue1
league_re = re.compile('[a-zA-Z0-9]+')

In [8]:
# 매 폴더를 돌며
for folder in folder_list:

    # 상위경로와 하위 경로를 병합할 변수
    sub_path = ''

    # 폴더명으로 새 경로를 생성
    sub_path = os.path.join(data_path,folder)

    # 시즌을 추출
    season = season_re.findall(folder)

    # 리그를 추출
    league = league_re.findall(folder)

    # 2020-2021의 형태로 시즌을 만들기
    season = season[0] + '-' + season[1]

    # 한 시즌의 모든 팀들의 데이터를 병합하고 전처리
    df_merged = data_merge(sub_path, season)
    
    # 포지션 모델을 이용해서 선수들의 포지션을 판정
    df_with_position = position_predictor(df_merged)
    
    # 포지션 판정된 데이터 전처리 및 변환
    df_clean = data_cleaner(df_with_position)
    
    # 폴더명과 2020-2021-Ligue1.csv의 형식으로 파일을 저장
    csv_path = target_path+'/'+season+'/'+ season +'-'+ league[2]+ '.csv'
    csv_file = open(csv_path,'w',newline='',encoding='utf-8')
    csvwriter = csv.writer(csv_file)
    csvwriter.writerow(['이름','시즌','골','도움','태클 성공률','태클','헤더 시도','헤더 성공%','경기 당 드리블','드리블','패스 시도','패스 %','기회 창출/90','Int/90','달린 거리/90분','Cr A','Cr C/A','슈팅 수','유효 슈팅 %','표준획득승점','포지션'])
    for i in range(len(df_clean)):
        csvwriter.writerow(df_clean.loc[i,['이름','시즌','골','도움','태클 성공률','태클','헤더 시도','헤더 성공%','경기 당 드리블','드리블','패스 시도','패스 %','기회 창출/90','Int/90','달린 거리/90분','Cr A','Cr C/A','슈팅 수','유효 슈팅 %','표준획득승점','포지션']].tolist())
    csv_file.close()
    
    # pyspark를 위한 df에 연결
    df_for_pyspark = pd.concat([df_for_pyspark,df_clean.loc[:,['태클','태클 성공률','헤더 시도','헤더 성공%','경기 당 드리블','드리블','패스 시도','패스 %','기회 창출/90','Int/90','달린 거리/90분','Cr A','Cr C/A','슈팅 수','유효 슈팅 %','표준획득승점','포지션']]])

In [10]:
# 포지션을 포함한 df를 저장할 폴더 생성
os.makedirs( ml_data_path+'/'+season,exist_ok=True)
csv_path = ml_data_path+'/'+season+'/'+ season + 'ML.csv'
csv_file = open(csv_path,'w',newline='',encoding='utf-8')
csvwriter = csv.writer(csv_file)


# pyspark에서 사용가능 하도록 영어 변수로 변경
csvwriter.writerow(['tkl','tkl_suc','header_att','header_suc','drib_per_g','drib','pass_att','pass_suc','CC','Int','Dist_covered','Cr_att','Cr_suc','shot','shot_on_t','gen_point','position'])

df_for_pyspark.reset_index(drop=True)

for i in range(len(df_for_pyspark)):
    csvwriter.writerow(df_for_pyspark.iloc[i,:].tolist())
csv_file.close()